In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("combine_text_img_prob_train.csv")
train_df.head()

,id,text_pred,label,image_pred
0,1115290475811037185,0.226079,0,0.49887
1,1108180634730745857,0.266702,0,0.49887
2,1064098809314230272,0.535388,0,0.49887
3,1113434967588188166,0.280910,0,0.49887
4,1114301234767368192,0.306098,0,0.49887


In [3]:
y_train = train_df['label']

In [4]:
X_train = train_df.drop(["id", "label"],axis = 1)
X_train.head()

,text_pred,image_pred
0,0.226079,0.49887
1,0.266702,0.49887
2,0.535388,0.49887
3,0.280910,0.49887
4,0.306098,0.49887


In [5]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [6]:
test_df = pd.read_csv("combine_text_img_prob_test.csv")
test_df.head()

,id,text_pred,label,image_pred
0,1115275629476372480,0.252771,0,0.49887
1,1114404944499625984,0.269186,0,0.49887
2,1107329660558471171,0.849918,0,0.49887
3,1107665629409099777,0.270317,0,0.49887
4,1117318148275961856,0.906745,1,0.49887


In [7]:
y_test = test_df['label']

In [8]:
y_test.head()

0    0
1    0
2    0
3    0
4    1
Name: label, dtype: int64

In [9]:
X_test = test_df.drop(["id", "label"],axis = 1)
X_test.head()

,text_pred,image_pred
0,0.252771,0.49887
1,0.269186,0.49887
2,0.849918,0.49887
3,0.270317,0.49887
4,0.906745,0.49887


In [11]:
X_test.shape

(10000, 2)

In [12]:
X_train.shape

(58954, 2)

In [13]:
y_train.shape

(58954,)

In [14]:
y_test.shape

(10000,)

In [17]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

# Create XGBoost classifier without specifying tree_method
xgb_model = XGBClassifier()

# Define parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2]
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)

# Fit the grid search to the data
grid_result = grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_result.best_params_
print("Best Parameters for XG Boost:", best_params)

# Train the model with the best parameters
best_model = XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

# Evaluate the model
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy for XG Boost:", accuracy)

# Predict probabilities for each class
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)

print("AUC Score for XG Boost:", auc_score)

Best Parameters for XG Boost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Test Accuracy for XG Boost: 0.6737
AUC Score for XG Boost: 0.7192830087713203


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score

# Create Logistic Regression classifier
logreg_model = LogisticRegression(solver='liblinear', random_state=42)

# Define parameter grid for grid search
logreg_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
}

# Create GridSearchCV object
logreg_grid_search = GridSearchCV(estimator=logreg_model, param_grid=logreg_param_grid, scoring='accuracy', cv=3, n_jobs=-1)

# Fit the grid search to the data
logreg_grid_result = logreg_grid_search.fit(X_train, y_train)

# Get the best parameters
best_logreg_params = logreg_grid_result.best_params_
print("Best Parameters for Logistic Regression:", best_logreg_params)

# Train the Logistic Regression model with the best parameters
best_logreg_model = LogisticRegression(solver='liblinear', random_state=42, **best_logreg_params)
best_logreg_model.fit(X_train, y_train)

# Evaluate the Logistic Regression model
y_logreg_pred = best_logreg_model.predict(X_test)
logreg_accuracy = accuracy_score(y_test, y_logreg_pred)

print("Test Accuracy for Logistic Regression:", logreg_accuracy)

# Predict probabilities for each class
y_logreg_pred_proba = best_logreg_model.predict_proba(X_test)[:, 1]

# Calculate the AUC score for Logistic Regression
logreg_auc_score = roc_auc_score(y_test, y_logreg_pred_proba)

print("AUC Score for Logistic Regression:", logreg_auc_score)

Best Parameters for Logistic Regression: {'C': 1}
Test Accuracy for Logistic Regression: 0.6756
AUC Score for Logistic Regression: 0.7197437887897515
